In [1]:
import re

def to_audio_link_format_single_verse(single_verse):
    '''
    >>> to_audio_link_format_single_verse('1 John 5:1\u201312')
    '1John.5'
    '''
    
    book_transformations_dict = {
        'Genesis': 'Gen',
        'Exodus': 'Exod',
        'Leviticus': 'Lev',
        'Numbers': 'Num',
        'Deuteronomy': 'Deut',
        'Joshua': 'Josh',
        'Judges': 'Judg',
        'Ruth': 'Ruth',
        '1 Samuel': '1Sam',
        '2 Samuel': '2Sam',
        '1 Kings': '1Kgs',
        '2 Kings': '2Kgs',
        '1 Chronicles': '1Chr',
        '2 Chronicles': '2Chr',
        'Ezra': 'Ezra',
        'Nehemiah': 'Neh',
        'Esther': 'Esth',
        'Job': 'Job',
        'Psalm': 'Ps',
        'Proverbs': 'Prov',
        'Ecclesiastes': 'Eccl',
        'Song of Solomon': 'Song',
        'Isaiah': 'Isa',
        'Jeremiah': 'Jer',
        'Lamentations': 'Lam',
        'Ezekiel': 'Ezek',
        'Daniel': 'Dan',
        'Hosea': 'Hos',
        'Joel': 'Joel',
        'Amos': 'Amos',
        'Obadiah': 'Obad',
        'Jonah': 'Jonah',
        'Micah': 'Mic',
        'Nahum': 'Nah',
        'Habakkuk': 'Hab',
        'Zephaniah': 'Zeph',
        'Haggai': 'Hag',
        'Zechariah': 'Zech',
        'Malachi': 'Mal',
        'Matthew': 'Matt',
        'Mark': 'Mark',
        'Luke': 'Luke',
        'John': 'John',
        'Acts': 'Acts',
        'Romans': 'Rom',
        '1 Corinthians': '1Cor',
        '2 Corinthians': '2Cor',
        'Galatians': 'Gal',
        'Ephesians': 'Eph',
        'Philippians': 'Phil',
        'Colossians': 'Col',
        '1 Thessalonians': '1Thess',
        '2 Thessalonians': '2Thess',
        '1 Timothy': '1Tim',
        '2 Timothy': '2Tim',
        'Titus': 'Titus',
        'Philemon': 'Phlm',
        'Hebrews': 'Heb',
        'James': 'Jas',
        '1 Peter': '1Pet',
        '2 Peter': '2Pet',
        '1 John': '1John',
        '2 John': '2John',
        '3 John': '3John',
        'Jude': 'Jude',
        'Revelation': 'Rev'
    }

    result = single_verse
    
    # chop off the verse
    result = re.sub(':.*$', '', result)
    
    # find the best key
    matching_keys = [key for key in book_transformations_dict.keys() if key in result]
    matching_keys.sort(key=len)
    if len(matching_keys) > 0:
        longest_key = matching_keys[-1]
    else:
        return result
       
    #perform substitution
    result = re.sub(longest_key, book_transformations_dict[longest_key], result)
    
    # replace space with .
    result = result.replace(' ', '.')
    
    return result
    

def to_audio_link_format(verses_together):
    '''
    >>> to_audio_link_format('Psalm 103; Genesis 17:1\u201313; Romans 2:17\u201329')
    'Ps.103,Gen.17,Rom.2'
    '''
    split_verses = [verse.strip() for verse in verses_together.split(';')]
    processed_verses = [to_audio_link_format_single_verse(verse) for verse in split_verses] 
    return ','.join(processed_verses)
    

def to_reading_link_format(verses_together):
    '''
    >>> to_reading_link_format('Psalm 103; Genesis 17:1\u201313; Romans 2:17\u2013end')
    'Psalm+103%3B+Genesis+17:1-13%3B+Romans+2:17-'
    '''
    result = verses_together.replace(' ', '+')
    result = result.replace('end', '')
    result = result.replace(';', '%3B')
    result = result.replace('\u2013','-') # remove em dash again
    return result


In [2]:
import csv
import json

json_dictionary = {}
json_dictionary_v2 = {}

def clean(input):
    input = input.replace('*', '')
    input = input.replace('-', '\u2013') # ensure using em dash
    return input

with open('./input/all_readings.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
         
        day_dictionary = {}
        day_dictionary['date_pretty'] = row['date_pretty']
        day_dictionary['day_description'] = row['day_description']
        
        cleaned_morning_readings = clean(row['morning_prayers_together'])
        day_dictionary['morning'] = {
            'readings' : [reading.strip() for reading in cleaned_morning_readings.split(';')],
            'reading_link': to_reading_link_format(cleaned_morning_readings),
            'audio_link': to_audio_link_format(cleaned_morning_readings)
        }
        
        cleaned_evening_readings = clean(row['evening_prayers_together'])
        day_dictionary['evening'] = {
            'readings' : [reading.strip() for reading in cleaned_evening_readings.split(';')],
            'reading_link': to_reading_link_format(cleaned_evening_readings),
            'audio_link': to_audio_link_format(cleaned_evening_readings)
        }
        
        json_dictionary[row['date']] = day_dictionary
        
print(json.dumps(json_dictionary, indent=4))

{
    "2023/12/03": {
        "date_pretty": "Sunday, December 3, 2023",
        "day_description": "1st Sunday of Advent",
        "morning": {
            "readings": [
                "Psalm 44",
                "Isaiah 2:1\u20135",
                "Luke 12:35\u201348"
            ],
            "reading_link": "Psalm+44%3B+Isaiah+2:1-5%3B+Luke+12:35-48",
            "audio_link": "Ps.44,Isa.2,Luke.12"
        },
        "evening": {
            "readings": [
                "Psalm 25",
                "Isaiah 1:1\u201320",
                "Matthew 21:1\u201313"
            ],
            "reading_link": "Psalm+25%3B+Isaiah+1:1-20%3B+Matthew+21:1-13",
            "audio_link": "Ps.25,Isa.1,Matt.21"
        }
    },
    "2023/12/04": {
        "date_pretty": "Monday, December 4, 2023",
        "day_description": "Week of 1st Sunday of Advent",
        "morning": {
            "readings": [
                "Psalm 50",
                "Isaiah 25:1\u20139",
                "Isaiah 25:1\

In [3]:
with open('./input/all_readings.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
         
        day_dictionary = {}
        day_dictionary['datePretty'] = row['date_pretty']
        day_dictionary['calendar'] = row['day_description']
        
        cleaned_morning_readings = clean(row['morning_prayers_together'])
        split_morning_readings = [reading.strip() for reading in cleaned_morning_readings.split(';')]
        morning_readings = []
        for reading in split_morning_readings:
            morning_reading = {
                'pretty': reading,
                'readQS': to_reading_link_format(reading),
                'audioQS': to_audio_link_format(reading)
            }
            morning_readings.append(morning_reading)
            
        day_dictionary['morning'] = {
            'readings' : morning_readings,
            'readQS': to_reading_link_format(cleaned_morning_readings),
            'audioQS': to_audio_link_format(cleaned_morning_readings)
        }
        
        cleaned_evening_readings = clean(row['evening_prayers_together'])
        split_evening_readings = [reading.strip() for reading in cleaned_evening_readings.split(';')]
        evening_readings = []
        for reading in split_evening_readings:
            evening_reading = {
                'pretty': reading,
                'readQS': to_reading_link_format(reading),
                'audioQS': to_audio_link_format(reading)
            }
            evening_readings.append(evening_reading)
            
        day_dictionary['evening'] = {
            'readings' : evening_readings,
            'readQS': to_reading_link_format(cleaned_evening_readings),
            'audioQS': to_audio_link_format(cleaned_evening_readings)
        }
        
        json_dictionary_v2[row['date']] = day_dictionary
        
print(json.dumps(json_dictionary_v2, indent=4))

{
    "2023/12/03": {
        "datePretty": "Sunday, December 3, 2023",
        "calendar": "1st Sunday of Advent",
        "morning": {
            "readings": [
                {
                    "pretty": "Psalm 44",
                    "readQS": "Psalm+44",
                    "audioQS": "Ps.44"
                },
                {
                    "pretty": "Isaiah 2:1\u20135",
                    "readQS": "Isaiah+2:1-5",
                    "audioQS": "Isa.2"
                },
                {
                    "pretty": "Luke 12:35\u201348",
                    "readQS": "Luke+12:35-48",
                    "audioQS": "Luke.12"
                }
            ],
            "readQS": "Psalm+44%3B+Isaiah+2:1-5%3B+Luke+12:35-48",
            "audioQS": "Ps.44,Isa.2,Luke.12"
        },
        "evening": {
            "readings": [
                {
                    "pretty": "Psalm 25",
                    "readQS": "Psalm+25",
                    "audioQS": "Ps.25"
  

In [4]:
import doctest
doctest.testmod(verbose=False)

TestResults(failed=0, attempted=3)

In [5]:
with open('./output/data.js', 'w') as outputfile:
    outputfile.write('class Data {\r')
    outputfile.write('  data = ' + json.dumps(json_dictionary, indent=4) + ';\r')
    outputfile.write('  getData(){return this.data;}\r')
    outputfile.write('}')

In [6]:
with open('./output/data.ts', 'w') as outputfile:
    outputfile.write('import { IReadingsForDay } from "./interfaces";\r\r')
    outputfile.write('const Data: { [dateKey: string]: IReadingsForDay } = ')
    outputfile.write(json.dumps(json_dictionary_v2, indent=4) + ';\r\r')
    outputfile.write('export default Data;')